Book Reference: page 191-207 of **Python for Data Analysis Book by Wes McKinney**

# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up **80% or more of an analyst’s time**

## Handling Missing Data
- For numeric data, pandas uses the floating-point value NaN (Not a number) to represent missing data. (This is called _sentinel value_ that can be easily detected.
- In statistics applications NA data may either be
    1. data that does not exist
    2. data that exists but was not observed through problems with data collection, for example
- When cleaning up data for analysis, it is often important to do **analysis on the missing data itself** to identify data collection problems or **potential biases** in the data caused by missing data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()
# On a Series, 
# dropna() returns the Series with only the non-null data and index values:

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
# data.dropna() is equivalent to
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

dropna by default drops any row containing a missing value in a DataFrame

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
     [NA, NA, NA], [NA, 6.5, 3.]])

In [10]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
cleaned = data.dropna()

In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1:`

In [14]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1, how='all') # column 4 is dropped

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the `thresh` argument:

In [16]:
df = pd.DataFrame(np.random.randn(7, 3))

In [17]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

In [18]:
df

,0,1,2
0,0.707638,NaN,NaN
1,-0.761852,NaN,NaN
2,-0.586471,NaN,0.635472
3,1.882848,NaN,-1.429640
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


In [19]:
df.dropna()

,0,1,2
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


In [20]:
df

,0,1,2
0,0.707638,NaN,NaN
1,-0.761852,NaN,NaN
2,-0.586471,NaN,0.635472
3,1.882848,NaN,-1.429640
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


In [21]:
df.dropna()

,0,1,2
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


In [22]:
df.dropna(thresh=2)

,0,1,2
2,-0.586471,NaN,0.635472
3,1.882848,NaN,-1.429640
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


### Filling in Missing Data

In [23]:
df.fillna(0)

,0,1,2
0,0.707638,0.000000,0.000000
1,-0.761852,0.000000,0.000000
2,-0.586471,0.000000,0.635472
3,1.882848,0.000000,-1.429640
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


In [24]:
df.fillna({1: 0.5, 2: 0}) # different fill value for each column

,0,1,2
0,0.707638,0.500000,0.000000
1,-0.761852,0.500000,0.000000
2,-0.586471,0.500000,0.635472
3,1.882848,0.500000,-1.429640
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


`fillna` returns a new object, but you can modify the existing object in-place

In [25]:
_ = df.fillna(0, inplace=True)

In [26]:
df

,0,1,2
0,0.707638,0.000000,0.000000
1,-0.761852,0.000000,0.000000
2,-0.586471,0.000000,0.635472
3,1.882848,0.000000,-1.429640
4,1.949750,-0.967942,0.751166
5,-0.090579,-0.018859,-0.233037
6,0.772055,-0.462395,-0.357645


In [27]:
# The same interpolation methods available for reindexing 
# can be used with fillna:
# ffill forward fill
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

In [28]:
df

,0,1,2
0,-0.829291,-1.092896,-0.669473
1,-1.806021,-0.669253,0.580983
2,0.095976,NaN,0.322806
3,0.464051,NaN,0.249778
4,-0.487570,NaN,NaN
5,-1.292458,NaN,NaN


In [29]:
df.fillna(method='ffill')

,0,1,2
0,-0.829291,-1.092896,-0.669473
1,-1.806021,-0.669253,0.580983
2,0.095976,-0.669253,0.322806
3,0.464051,-0.669253,0.249778
4,-0.487570,-0.669253,0.249778
5,-1.292458,-0.669253,0.249778


In [30]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.829291,-1.092896,-0.669473
1,-1.806021,-0.669253,0.580983
2,0.095976,-0.669253,0.322806
3,0.464051,-0.669253,0.249778
4,-0.487570,NaN,0.249778
5,-1.292458,NaN,0.249778


With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [31]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [32]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

In [33]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
     'k2': [1, 1, 2, 3, 3, 4, 4]})

In [34]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [35]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [36]:
# drop_duplicates returns a DataFrame where the duplicated array is False:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


---

In [37]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [38]:
data.drop_duplicates(['k1']) # filter duplicates only based on the 'k1' column

,k1,k2,v1
0,one,1,0
1,two,1,1


In [39]:
# duplicated and drop_duplicates by default keep the first observed value combination.
# as shown from prev example

#Passing keep='last' will return the last one:
data.drop_duplicates(['k1', 'k2'], keep='last') # row with index '5' is dropped

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

In [40]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
     'Pastrami', 'corned beef', 'Bacon',
     'pastrami', 'honey ham', 'nova lox'],
     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [41]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food
came from. Let’s write down a mapping of each distinct meat type to the kind of
animal:

In [42]:
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

In [43]:
lowercased = data['food'].str.lower()

In [44]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [45]:
data['animal'] = lowercased.map(meat_to_animal) # map func

In [46]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [47]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

### Replacing Values

In [48]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [49]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be **sentinel values** for missing data

In [50]:
data.replace(-999, np.nan) # use inplace = True if you want to replace the values inplace

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [51]:
# If you want to replace multiple values at once, 
# you instead pass a list and then the substitute value
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [52]:
# To use a different replacement for each value, pass a list of substitutes:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [53]:
# other method 
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [54]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four'])

In [55]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [56]:
# Like a Series, the axis indexes have a map method
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [57]:
data.index = data.index.map(transform)

In [58]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [59]:
# If you want to create a transformed version of a dataset 
# without modifying the original, a useful method is rename:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, `rename` can be used in conjunction with a dict-like object providing new values
for a subset of the axis labels:

In [60]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


---

In [61]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
    index=['Ohio', 'Colorado', 'New York'],
    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [62]:
data.rename(index={'Ohio': 'INDIANA'}, inplace=True)

In [63]:
data

,one,two,three,four
INDIANA,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets:

In [64]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [65]:
# Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, 
# and finally 61 and older. To
# do so, you have to use cut, a function in pandas:
bins = [18, 25, 35, 60, 100]

In [66]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [67]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [68]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [69]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

---

In [70]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

---

In [71]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

---

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute
equal-length bins based on the minimum and maximum values in the data.
Consider the case of some uniformly distributed data chopped into fourths:

In [72]:
data = np.random.randint(20, size=20)
data

array([10,  0, 14, 14,  0,  9, 19,  0,  2, 19, 12, 10, 10, 13,  1, 13,  9,
        1,  0,  3])

In [73]:
pd.cut(data, 4, precision=0)

[(10.0, 14.0], (-0.0, 5.0], (10.0, 14.0], (10.0, 14.0], (-0.0, 5.0], ..., (10.0, 14.0], (5.0, 10.0], (-0.0, 5.0], (-0.0, 5.0], (-0.0, 5.0]]
Length: 20
Categories (4, interval[float64]): [(-0.0, 5.0] < (5.0, 10.0] < (10.0, 14.0] < (14.0, 19.0]]

---

In [74]:
data = np.random.randn(1000) # Normally distributed

In [75]:
cats = pd.qcut(data, 4) # roughly equal-size bins

In [76]:
cats

[(-0.643, 0.0879], (0.0879, 0.767], (0.0879, 0.767], (0.0879, 0.767], (0.0879, 0.767], ..., (0.767, 3.313], (0.0879, 0.767], (0.767, 3.313], (-0.643, 0.0879], (-2.915, -0.643]]
Length: 1000
Categories (4, interval[float64]): [(-2.915, -0.643] < (-0.643, 0.0879] < (0.0879, 0.767] < (0.767, 3.313]]

In [77]:
pd.value_counts(cats)

(0.767, 3.313]      250
(0.0879, 0.767]     250
(-0.643, 0.0879]    250
(-2.915, -0.643]    250
dtype: int64

---

In [78]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) # Similar to cut you can pass your own quantiles

[(-1.29, 0.0879], (0.0879, 1.339], (0.0879, 1.339], (0.0879, 1.339], (0.0879, 1.339], ..., (1.339, 3.313], (0.0879, 1.339], (0.0879, 1.339], (-1.29, 0.0879], (-1.29, 0.0879]]
Length: 1000
Categories (4, interval[float64]): [(-2.915, -1.29] < (-1.29, 0.0879] < (0.0879, 1.339] < (1.339, 3.313]]

### Detecting and Filtering Outliers

In [79]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.018188,0.008642,-0.032326,0.006992
std,1.008110,1.020089,1.019047,0.995028
min,-3.524788,-3.237069,-2.851837,-3.071513
25%,-0.697681,-0.689573,-0.746434,-0.635254
50%,-0.021656,0.024031,-0.002235,-0.002100
75%,0.701880,0.735725,0.618008,0.699768
max,2.590596,3.054263,4.365329,2.723357


In [80]:
data.head()

,0,1,2,3
0,-0.043667,-0.325040,0.348038,0.505116
1,0.916318,1.013659,-0.619034,-0.473803
2,1.205966,0.501570,1.849865,0.165021
3,0.094345,0.909519,1.366694,0.258502
4,-0.999022,0.006167,0.536148,-0.282926


In [81]:
col = data[2] # get column 2
col.head()

0    0.348038
1   -0.619034
2    1.849865
3    1.366694
4    0.536148
Name: 2, dtype: float64

Suppose you wanted to find values in one of the columns exceeding 3 in absolute
value:

In [82]:
col[np.abs(col) > 3]

171    3.490806
325    3.604196
639    4.365329
Name: 2, dtype: float64

In [83]:
# To select all rows having a value exceeding 3 or –3, 
# you can use the any method on a boolean DataFrame:
data[(np.abs(data) > 3).any(1)]  # any == Return whether any element is True over requested axis

,0,1,2,3
171,0.146175,0.723994,3.490806,-0.333566
318,1.766859,-3.237069,0.703324,1.520062
325,0.742386,0.784822,3.604196,1.300593
348,1.554357,0.625910,0.004374,-3.013065
403,-3.524788,-0.057789,-1.137459,0.187076
639,0.647778,0.894967,4.365329,-0.880368
650,-0.457444,-3.207747,-0.539727,-0.098993
765,-0.382593,3.021669,0.027869,0.471980
787,0.838825,0.957855,-0.598250,-3.071513
855,-0.064386,3.054263,0.266820,-1.329156


In [84]:
# Values can be set based on these criteria. 
# Here is code to cap values outside the interval –3 to 3:
data[np.abs(data) > 3] = np.sign(data) * 3 
# np.sign returns sign of the num (if positive or negative) -1 or 1
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.017663,0.009010,-0.034787,0.007135
std,1.006418,1.018501,1.010466,0.994593
min,-3.000000,-3.000000,-2.851837,-3.000000
25%,-0.697681,-0.689573,-0.746434,-0.635254
50%,-0.021656,0.024031,-0.002235,-0.002100
75%,0.701880,0.735725,0.618008,0.699768
max,2.590596,3.000000,3.000000,2.723357


In [85]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,1.0
1,1.0,1.0,-1.0,-1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,-1.0,1.0,1.0,-1.0


### Permutation and Random Sampling
- Permutation / randomly reordering

In [86]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [87]:
sampler = np.random.permutation(5)
sampler

array([4, 3, 1, 0, 2])

That array can then be used in iloc-based indexing or the equivalent take function:

In [88]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [89]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11


To select a random subset without replacement, you can use the `sample` method on
Series and DataFrame:

In [90]:
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3


To generate a sample with replacement (to allow repeat choices), pass replace=True
to sample:

In [93]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [94]:
draws = choices.sample(n=10, replace=True)

In [96]:
draws

1    7
3    6
0    5
0    5
1    7
2   -1
0    5
4    4
0    5
4    4
dtype: int64

## Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications
is converting a categorical variable into a “dummy” or “indicator” matrix.

In [100]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
     'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [99]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


---

In [101]:
dummies = pd.get_dummies(df['key'], prefix='key') #adding prefix to the columns in the indicator DF

In [102]:
df_with_dummy = df[['data1']].join(dummies)

In [103]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


---

In [105]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
     header=None, names=mnames, engine='python')

In [106]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling. First, we
extract the list of unique genres in the dataset:

In [107]:
all_genres = []

In [108]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [109]:
genres = pd.unique(all_genres)

In [110]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [112]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [113]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [114]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [115]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [116]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [118]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [119]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

---
A useful recipe for statistical applications is to combine get_dummies with a discretization
function like cut:

In [127]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [128]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values,bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

In [129]:
val = 'a,b, guido'
val.split(',') # method 1

['a', 'b', ' guido']

In [130]:
pieces = [x.strip() for x in val.split(',')] # method 2

In [131]:
pieces

['a', 'b', 'guido']

---

In [132]:
first, second, third = pieces

In [133]:
first + '::' + second + '::' + third # method 1

'a::b::guido'

In [134]:
'::'.join(pieces) # method 2

'a::b::guido'

---

In [136]:
'guido' in val

True

In [137]:
val.index(',')

1

In [138]:
val.find(':')

-1

In [140]:
val.index(':')
# index raises an exception if the
# string isn’t found (versus returning –1 in val.find):

ValueError: substring not found

In [141]:
val.count(',')

2

In [142]:
val.replace(',', '::')

'a::b:: guido'

In [143]:
val.replace(',', '')

'ab guido'

## Regular Expressions

`re` module functions:
1. pattern matching
2. substitution
3. splitting

In [146]:
import re
text = "foo bar\t baz \tqux"

In [147]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

When you call re.split('\s+', text), the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with re.compile, forming a reusable regex object:

In [148]:
regex = re.compile('\s+')

In [149]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [150]:
regex.findall(text)

[' ', '\t ', ' \t']

To avoid unwanted escaping with \ in a regular expression, use raw
string literals like `r'C:\x'` instead of the equivalent `'C:\\x'`.

---

- re.match - only matches at the beginning of the string
- re.search - returns only the first match
- re.findall - returns all matches in a string

---

In [151]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [152]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [153]:
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [154]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

---

In [155]:
m = regex.search(text)

In [156]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [157]:
text[m.start():m.end()]

'dave@google.com'

---

In [159]:
# regex.match returns None, as it only will match 
# if the pattern occurs at the start of the string
print(regex.match(text))

None


---

In [161]:
# sub will return a new string with occurrences of the pattern replaced by the
# a new string
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



---

In [162]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [163]:
m = regex.match('wesm@bright.net')

In [164]:
m.groups()

('wesm', 'bright', 'net')

In [165]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [166]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas

In [167]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
     'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [168]:
data = pd.Series(data)

In [169]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [170]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [172]:
data.str.contains('gmail') # skip NA values

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [173]:
 pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [174]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

#### Ways to do vectorized element retrieval

In [175]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [176]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [178]:
matches = data.str.extract(pattern, flags=re.IGNORECASE)
matches

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


In [179]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object